<a href="https://colab.research.google.com/github/arai2878/Hello-World/blob/master/WorldDatameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)


In [5]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [6]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,\nNorth America\n,"39,485,100","+1,807","885,053",+58,"32,150,083","+1,411","6,449,964","14,869",,,,,,North America,\n,,,,,
1,,\nAsia\n,"49,271,602","+56,173","649,035",+941,"44,435,141","+64,596","4,187,426","32,698",,,,,,Asia,\n,,,,,
2,,\nSouth America\n,"27,752,722","+1,466","754,165",+55,"24,985,607","+1,141","2,012,950","28,996",,,,,,South America,\n,,,,,
3,,\nEurope\n,"46,214,885","+15,743","1,061,610",+579,"42,736,143","+39,860","2,417,132","18,448",,,,,,Europe,\n,,,,,
4,,\nAfrica\n,"4,800,209",+152,"128,751",+7,"4,325,227",+221,"346,231","2,803",,,,,,Africa,\n,,,,,


In [7]:
df_bs.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
219,212,Saint Pierre Miquelon,25,,,,25,,0,,"4,334",,"5,736","994,280","5,769",North America,231,,1,,,
220,213,Montserrat,20,,1,,19,,0,,"4,004",200,"1,408","281,882","4,995",North America,250,"4,995",4,,,
221,214,Solomon Islands,20,,,,20,,0,,28,,"4,500","6,412","701,811",Australia/Oceania,"35,091",,156,,,
222,215,Western Sahara,10,,1,,8,,1,,16,2,,,"610,353",Africa,"61,035","610,353",,,,2
223,216,MS Zaandam,9,,2,,7,,0,,,,,,,,,,,,,


In [8]:

# drop unwanted rows
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,USA,"33,896,660",,"604,087",,"27,502,255",,"5,790,318","6,980","101,873","1,816","473,304,856","1,422,476","332,733,197",North America,10,551,1
1,India,"26,752,447",+766,"303,751",,"23,728,011","+7,092","2,720,685","8,944","19,218",218,"330,536,064","237,439","1,392,086,685",Asia,52,"4,583",4
2,Brazil,"16,083,573",,"449,185",,"14,492,167",,"1,142,221","8,318","75,190","2,100","49,013,934","229,138","213,906,072",South America,13,476,4
3,France,"5,603,666",,"108,596",,"5,199,240",,"295,830","3,515","85,680","1,660","82,691,176","1,264,344","65,402,419",Europe,12,602,1
4,Turkey,"5,186,487",,"46,268",,"5,024,313",,"115,906","1,843","60,912",543,"52,377,678","615,144","85,147,010",Asia,16,"1,840",2


In [9]:
# rename columns
df_bs.columns = ['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n',
       'Population', 'Continent','1 Caseevery X ppl','1 Deathevery X ppl','1 Testevery X ppl']

# rearrange and subselect columns
df_bs = df_bs[['Country,Other', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n',
       'Population', 'Continent','1 Caseevery X ppl','1 Deathevery X ppl','1 Testevery X ppl' ]]
# first few rows
df_bs.head()

,"Country,Other",Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,USA,North America,"332,733,197","33,896,660",,"604,087",,"27,502,255",,"5,790,318","6,980","101,873","1,816","473,304,856","1,422,476","332,733,197",North America,10,551,1
1,India,Asia,"1,392,086,685","26,752,447",+766,"303,751",,"23,728,011","+7,092","2,720,685","8,944","19,218",218,"330,536,064","237,439","1,392,086,685",Asia,52,"4,583",4
2,Brazil,South America,"213,906,072","16,083,573",,"449,185",,"14,492,167",,"1,142,221","8,318","75,190","2,100","49,013,934","229,138","213,906,072",South America,13,476,4
3,France,Europe,"65,402,419","5,603,666",,"108,596",,"5,199,240",,"295,830","3,515","85,680","1,660","82,691,176","1,264,344","65,402,419",Europe,12,602,1
4,Turkey,Asia,"85,147,010","5,186,487",,"46,268",,"5,024,313",,"115,906","1,843","60,912",543,"52,377,678","615,144","85,147,010",Asia,16,"1,840",2


In [10]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'


In [11]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country,Other'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country,Other'].unique()

array(['Réunion', 'French Guiana', 'Mayotte', 'French Polynesia',
       'Guadeloupe', 'Curaçao', 'Martinique', 'Aruba', 'CAR', 'Gibraltar',
       'Channel Islands', 'Turks and Caicos', 'Sint Maarten',
       'St. Vincent Grenadines', 'Saint Martin', 'Caribbean Netherlands',
       'Isle of Man', 'St. Barth', 'Diamond Princess', 'Faeroe Islands',
       'Cayman Islands', 'Wallis and Futuna', 'British Virgin Islands',
       'Brunei ', 'New Caledonia', 'Anguilla', 'Falkland Islands',
       'Macao', 'Saint Pierre Miquelon', 'Montserrat', 'MS Zaandam'],
      dtype=object)

In [12]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

In [13]:
# first few rows
df_bs.head()

,"Country,Other",Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,WHO Region
0,USA,NorthAmerica,332733197,33896660,NaN,604087,NaN,27502255,NaN,5790318,6980,101873,1816,473304856,1422476,332733197,NorthAmerica,10,551,1,Americas
1,India,Asia,1392086685,26752447,766,303751,NaN,23728011,7092,2720685,8944,19218,218,330536064,237439,1392086685,Asia,52,4583,4,South-EastAsia
2,Brazil,SouthAmerica,213906072,16083573,NaN,449185,NaN,14492167,NaN,1142221,8318,75190,2100,49013934,229138,213906072,SouthAmerica,13,476,4,Americas
3,France,Europe,65402419,5603666,NaN,108596,NaN,5199240,NaN,295830,3515,85680,1660,82691176,1264344,65402419,Europe,12,602,1,Europe
4,Turkey,Asia,85147010,5186487,NaN,46268,NaN,5024313,NaN,115906,1843,60912,543,52377678,615144,85147010,Asia,16,1840,2,Europe


In [14]:
# first few rows
df_bs.tail()

,"Country,Other",Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,WHO Region
211,Saint Pierre Miquelon,NorthAmerica,5769,25,NaN,NaN,NaN,25,NaN,0,NaN,4334,NaN,5736,994280,5769,NorthAmerica,231,NaN,1,NaN
212,Montserrat,NorthAmerica,4995,20,NaN,1,NaN,19,NaN,0,NaN,4004,200,1408,281882,4995,NorthAmerica,250,4995,4,NaN
213,Solomon Islands,Australia/Oceania,701811,20,NaN,NaN,NaN,20,NaN,0,NaN,28,NaN,4500,6412,701811,Australia/Oceania,35091,NaN,156,WesternPacific
214,Western Sahara,Africa,610353,10,NaN,1,NaN,8,NaN,1,NaN,16,2,NaN,NaN,610353,Africa,61035,610353,NaN,Africa
215,MS Zaandam,NaN,NaN,9,NaN,2,NaN,7,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:

# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)

# New Section